<a href="https://colab.research.google.com/github/toinnn/Chat_Bot/blob/master/Chat_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
Git_Dir    = "/content/drive/MyDrive/Github_Dir/Chat_Bot"
Neural_Dir = "Machine-Learning__Deep-Learning"
with open("/content/drive/MyDrive/Github_Dir/acess_Token_Git.txt","r") as file:
    acess_Token_Git = file.read()
Git_Path   = "https://"+ acess_Token_Git + "@github.com/toinnn/" + Neural_Dir + ".git"
Git_CB_Path= "https://"+ acess_Token_Git + "@github.com/toinnn/" + "Chat_Bot" + ".git"

In [ ]:
!git clone "{Git_Path}" ./temp/Machine-Learning__Deep-Learning
!git clone "{Git_CB_Path}" ./temp/Chat_Bot

!mv ./temp/* "{Git_Dir}"
!rm -rf ./temp 

!rsync -aP "{Git_Dir}"/*  ./
#!ln -s "/content/drive/MyDrive/Github_Dir/Chat_Bot" + Neural_Dir NLP

In [56]:
class counter():
    def __init__(self):
        self.ctd = 0
    def add(self):
        self.ctd += 1
ctd = counter()
# add = lambda : ctd+=1
def charNgram(text : str , n : int ) -> list:
    var = {}
    text = text.split()
    ctd.add()
    for word in text :
        for i in range(0 ,len(word) -n + 1 , n):#n
            var.update({word[i : i + j] : None for j in range(n+1) })
        var.update({ word[- (len(word)%n):] : None})

    print(ctd.ctd)
    # var += [text[- (len(text)%n):]]
    print(tuple(var.keys()))
    print("funciono   muito isso".split())  
charNgram("cincocinco" , 4 )

1
('', 'c', 'ci', 'cin', 'cinc', 'o', 'oc', 'oci', 'ocin', 'co')
['funciono', 'muito', 'isso']


In [ ]:
%cd "/content/{Neural_Dir}"
from BiLSTM import BiLSTM ,BiLSTM_Attention
%cd /content/Chat_Bot





print("Rodei 123")


/content/Machine-Learning__Deep-Learning
teste
/content/Chat_Bot
Rodei 123


In [ ]:
# path = "/content/drive/MyDrive/Colab Notebooks/Chat_Bot.ipynb"
# !rsync -aP "{path}" /content/Chat_Bot

# !git clone "{Git_CB_Path}" ./temp
# !rsync -aP /content/Chat_Bot/* ./temp
# %cd ./temp
# !git add -u
# !git commit -m "update"
# !git config user.email "limaalyson@hotmail.com"
# !git config user.name "Alyson_Google_Colab"
# !git push origin master
# %cd /content
# !rm -rf ./temp


sending incremental file list
Chat_Bot.ipynb
          4,698 100%    0.00kB/s    0:00:00 (xfr#1, to-chk=0/1)
Cloning into './temp'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 12 (delta 1), reused 6 (delta 1), pack-reused 0
Unpacking objects: 100% (12/12), done.
sending incremental file list
Chat_Bot.ipynb
          4,698 100%    0.00kB/s    0:00:00 (xfr#1, to-chk=1/2)
README.md
             10 100%    9.77kB/s    0:00:00 (xfr#2, to-chk=0/2)
/content/temp
[master 0b8e7fd] update
 1 file changed, 1 insertion(+), 1 deletion(-)
 rewrite Chat_Bot.ipynb (95%)
Counting objects: 3, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 663 bytes | 663.00 KiB/s, done.
Total 3 (delta 1), reused 0 (delta 0)
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/toinnn/Chat_Bot.git
   9aed345..0b8e7f